# 算法精粹：经典计算机科学问题的 Python 实现

## 第一章 几个小问题

### 1.1 斐波那契序列
**斐波那契序列**（Fibonacci sequence）是一系列数字，其中除第1个和第2个数字之外，其他数字都是前两个数字之和：

$$0, 1, 1, 2, 3, 5, 8, 13, 21, …$$

在此序列中，第1个斐波那契数是0。第4个斐波那契数是2。后续任一斐波那契数n的值可用以下公式求得：

$$fib(n) = fib(n − 1) + fib(n − 2)$$

> 由于Jupyter无法捕捉**RecursionError**: maximum recursion depth exceeded(递归调用超出限制错误)
> 
> 会造成kernel挂掉重启, 下面第一种递归调用代码请复制到本地IDE环境执行

``` python
def fib1(n: int) -> int:
    return fib1(n - 1) + fib1(n - 2)
```

执行结果会报错：`RecursionError: maximum recursion depth exceeded`

**解释**：函数未设置下限，造成无限递归调用。

参考[Stackoverflow](https://stackoverflow.com/questions/3323001/what-is-the-maximum-recursion-depth-in-python-and-how-to-increase-it)，可以手动调节recursion depth： `sys.setrecursionlimit(3600)`

下面有关递归**调用次数**的[推导](https://blog.csdn.net/qq_43411555/article/details/88400990), [代码](https://blog.csdn.net/qq_46049116/article/details/103630603)

In [22]:
from time import perf_counter as tic
def fib2(n: int) -> int:
    global count
    if n < 2:
        count=count + 2
        return n
    return fib2(n - 1) + fib2(n - 2)

for i in range(2,31):
    count = -1
    st = tic()
    print(fib2(i),"---->耗时: {:5f}s".format(tic() - st), f"计算fab({i})调用{count}次")

# n大于30就容易挂掉, 需做如下改动
#from sys import setrecursionlimit
#setrecursionlimit(9000)
#st = tic()
#print(fib2(40),"---->耗时: {}s".format(tic() - st))
#setrecursionlimit(1000) # 还原默认值：1000

1 ---->耗时: 0.000003s 计算fab(2)调用3次
2 ---->耗时: 0.000005s 计算fab(3)调用5次
3 ---->耗时: 0.000005s 计算fab(4)调用9次
5 ---->耗时: 0.000006s 计算fab(5)调用15次
8 ---->耗时: 0.000009s 计算fab(6)调用25次
13 ---->耗时: 0.000015s 计算fab(7)调用41次
21 ---->耗时: 0.000023s 计算fab(8)调用67次
34 ---->耗时: 0.000037s 计算fab(9)调用109次
55 ---->耗时: 0.000060s 计算fab(10)调用177次
89 ---->耗时: 0.000097s 计算fab(11)调用287次
144 ---->耗时: 0.000157s 计算fab(12)调用465次
233 ---->耗时: 0.000293s 计算fab(13)调用753次
377 ---->耗时: 0.000417s 计算fab(14)调用1219次
610 ---->耗时: 0.000714s 计算fab(15)调用1973次
987 ---->耗时: 0.001119s 计算fab(16)调用3193次
1597 ---->耗时: 0.002088s 计算fab(17)调用5167次
2584 ---->耗时: 0.003337s 计算fab(18)调用8361次
4181 ---->耗时: 0.006361s 计算fab(19)调用13529次
6765 ---->耗时: 0.007771s 计算fab(20)调用21891次
10946 ---->耗时: 0.018308s 计算fab(21)调用35421次
17711 ---->耗时: 0.019255s 计算fab(22)调用57313次
28657 ---->耗时: 0.030712s 计算fab(23)调用92735次
46368 ---->耗时: 0.050106s 计算fab(24)调用150049次
75025 ---->耗时: 0.078581s 计算fab(25)调用242785次
121393 ---->耗时: 0.133462s 计算fab(26)调用392835次
196418 ---->耗时: 0

In [23]:
from time import perf_counter as tic
from typing import Dict
# 字典缓存
memo: Dict[int, int] = {0: 0, 1: 1}

def fib3(n: int) -> int:
    if n not in memo: 
        memo[n] = fib3(n - 1) + fib3(n - 2)
    return memo[n]

st = tic()
print(fib3(40),"---->耗时: {:5f}s".format(tic() - st),f"---->字典长度：{len(memo)}")


102334155 ---->耗时: 0.000270s ---->字典长度：41


In [18]:
from time import perf_counter as tic
from functools import lru_cache

#python内置自动缓存装饰器
@lru_cache(maxsize=None)
def fib4(n: int) -> int:  # same definition as fib2()
    if n < 2:  # base case
        return n
    return fib4(n - 2) + fib4(n - 1)  # recursive case

st = tic()
print(fib4(50),"---->耗时: {:5f}s".format(tic() - st))

12586269025 ---->耗时: 0.000226s


In [17]:
from time import perf_counter as tic
#迭代法
def fib5(n: int) -> int:
    if n == 0: return n  # special case
    last: int = 0  # fib(0)
    next: int = 1  # fib(1)
    for _ in range(1, n):
        last, next = next, last + next
    return next
st=tic()
print(fib5(50),"---->耗时: {:5f}s".format(tic() - st))

12586269025 ---->耗时: 0.000113s


### 1.2 简单的压缩


In [47]:
class CompressedGene:
    def __init__(self, gene: str) -> None:
        self._compress(gene)
        
    def _compress(self, gene: str) -> None:
        self.bit_string: int = 1  # start with sentinel
        for nucleotide in gene.upper():
            self.bit_string <<= 2  # shift left two bits
            if nucleotide == "A":  # change last two bits to 00
                self.bit_string |= 0b00
            elif nucleotide == "C":  # change last two bits to 01
                self.bit_string |= 0b01
            elif nucleotide == "G":  # change last two bits to 10
                self.bit_string |= 0b10
            elif nucleotide == "T":  # change last two bits to 11
                self.bit_string |= 0b11
            else:
                raise ValueError("Invalid Nucleotide:{}".format(nucleotide))
            
    def decompress(self) -> str:
        gene: str = ""
        for i in range(0, self.bit_string.bit_length() - 1, 2):  # -1 to exclude sentinel
            bits: int = self.bit_string >> i & 0b11  # get just 2 relevant bits
            if bits == 0b00:  # A
                gene += "A"
            elif bits == 0b01:  # C
                gene += "C"
            elif bits == 0b10:  # G
                gene += "G"
            elif bits == 0b11:  # T
                gene += "T"
            else:
                 raise ValueError("Invalid bits:{}".format(bits))
        return gene[::-1]  # [::-1] reverses string by slicing backward
    def __str__(self) -> str:  # string representation for pretty printing
        return self.decompress()

from sys import getsizeof
original: str = "TAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATATAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATA" * 100
print(f"original is {getsizeof(original)} bytes")
compressed: CompressedGene = CompressedGene(original)  # compress
print(f"compressed is {getsizeof(compressed.bit_string)} bytes")
print(str(compressed)[:90])  # decompress
print("original and decompressed are the same: {}".format(original == compressed.decompress()))

original is 8649 bytes
compressed is 2320 bytes
TAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATATAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATATAGG
original and decompressed are the same: True


### 1.3 牢不可破的加密

In [48]:
from secrets import token_bytes
from typing import Tuple

def random_key(length: int) -> int:
    # generate length random bytes
    tb: bytes = token_bytes(length)
    # convert those bytes into a bit string and return it
    return int.from_bytes(tb, "big")

def encrypt(original: str) -> Tuple[int, int]:
    original_bytes: bytes = original.encode()
    dummy: int = random_key(len(original_bytes))
    original_key: int = int.from_bytes(original_bytes, "big")
    encrypted: int = original_key ^ dummy  # XOR
    return dummy, encrypted

def decrypt(key1: int, key2: int) -> str:
    decrypted: int = key1 ^ key2  # XOR
    temp: bytes = decrypted.to_bytes((decrypted.bit_length()+ 7) // 8, "big")
    return temp.decode()

key1, key2 = encrypt("One Time Pad!")
result: str = decrypt(key1, key2)
print(result)

One Time Pad!


### 1.4 计算$\pi$

数学意义重大的$\pi$（3.14159…）用很多公式都可以推导出来，其中最简单的公式之一就是莱布尼茨公式。它断定以下无穷级数的收敛值等于$\pi$：

$$\pi = \dfrac{4}{1} − \dfrac{4}{3} + \dfrac{4}{5} − \dfrac{4}{7} + \dfrac{4}{9} − \dfrac{4}{11}…$$

请注意，以上无穷级数的分子保持为4，而分母则每次递增2，并且对每一项的操作是加法和减法交替出现。

In [26]:
from time import perf_counter as tic
def calculate_pi(n_terms: int) -> float:
    numerator: float = 4.0
    denominator: float = 1.0
    operation: float = 1.0
    pi: float = 0.0
    for _ in range(n_terms):
        pi += operation * (numerator / denominator)
        denominator += 2.0
        operation *= -1.0
    return pi

start = tic()
print(calculate_pi(6000000),"---->耗时: {:5f}s".format(tic() - start))

3.1415924869231824 ---->耗时: 2.113931s


### 1.5 汉诺塔


In [34]:
from time import perf_counter as tic
from typing import TypeVar, Generic, List
T = TypeVar('T')

class Stack(Generic[T]):
    def __init__(self) -> None:
        self._container: List[T] = []

    def push(self, item: T) -> None:
        self._container.append(item)

    def pop(self) -> T:
        return self._container.pop()

    def __repr__(self) -> str:
        return repr(self._container)
    
num_discs: int = 22
tower_a: Stack[int] = Stack()
tower_b: Stack[int] = Stack()
tower_c: Stack[int] = Stack()
for i in range(1, num_discs + 1):
    tower_a.push(i)
    
def hanoi(begin: Stack[int], end: Stack[int], temp: Stack[int], n: int) -> None:
    if n == 1:
        end.push(begin.pop())
    else:
        hanoi(begin, temp, end, n - 1)
        hanoi(begin, end, temp, 1)
        hanoi(temp, end, begin, n - 1)

start = tic()
hanoi(tower_a, tower_c, tower_b, num_discs)
print(tower_a)
print(tower_b)
print(tower_c)
print("---->耗时: {:5f}s".format(tic() - start))

[]
[]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]
---->耗时: 4.519683s
